<a href="https://www.kaggle.com/code/neloybarman018/data-cleaning-eda?scriptVersionId=157658463" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Necessary Imports**

In [1]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# **Fetching CSV Files**

In [2]:
# base_path = "/gdrive/MyDrive/Daraz 11.11 Top Sale Data Analysis/"
base_path = "/kaggle/input/daraz-11-11-top-selling-product-data/categorical_data"

In [3]:
csv_files = os.listdir(base_path)
print(f"Aavailable Files: {csv_files}")

Aavailable Files: ['electronic_accessories.csv', 'mother_baby.csv', 'automotive_motorbike.csv', 'men_boys_fashion.csv', 'sports_fitness.csv', 'tv_home_appliances.csv', 'health_beauty.csv', 'electronic_devices.csv', 'home_lifestyle.csv', 'watches_bags_jewellery.csv', 'groceries.csv', 'women_girls_fashion.csv']


# **Preparing the Product Data Frame**

In [4]:
data_frame = pd.DataFrame()

In [5]:
for file in csv_files:
  file_path = f"{base_path}/{file}"
  df = pd.read_csv(f"{file_path}")
  data_frame = pd.concat([data_frame, df], ignore_index=True)

# **Viewing the DataFrame**

In [6]:
data_frame.head()

,Category,SubCategory,Title,Original Price,Discount Price,Discount,Seller Name,Number of Ratings,Positive Seller Ratings,Ship On Time,Chat Response Rate,Delivery Type,Cash On Delivery,Flagship Store
0,Electronic Accessories,https://www.daraz.com.bd/audio/,QKZ DM10 Zinc Alloy HiFi Earphone In Ear Earphone,625,299,52.0,Time Square,4208,85,0,79,Standard Delivery,Cash on Delivery Available,Yes
1,Electronic Accessories,https://www.daraz.com.bd/audio/,Uiisii HM 12 Super Bass Stereo In-Ear Earphone...,520,229,56.0,Subaita Dreamland,579,82,0,98,Free Delivery,Cash on Delivery Available,No
2,Electronic Accessories,https://www.daraz.com.bd/audio/,Uiisii C100 Super Bass Stereo In Ear Headphone...,424,199,53.0,Time Square,2714,85,0,79,Standard Delivery,Cash on Delivery Available,Yes
3,Electronic Accessories,https://www.daraz.com.bd/audio/,Orignal Creative High Bass Audio Earphone with...,299,125,58.0,Rimjhim Shop,627,81,0,92,Free Delivery,Cash on Delivery Available,No
4,Electronic Accessories,https://www.daraz.com.bd/audio/,Vivo In Year Yearphone is the best sound quali...,270,99,63.0,Prima Store,374,84,0,96,Free Delivery,Cash on Delivery Available,No


# **Viewing DataFrame Shape**

In [7]:
print(f"Number of Rows: {data_frame.shape[0]}")
print(f"Number of Columns: {data_frame.shape[1]}")

Number of Rows: 13747
Number of Columns: 14


## **Checking the Columns**

In [8]:
columns = list(data_frame.columns)
print(f"Available Columns: {columns}")

Available Columns: ['Category', 'SubCategory', 'Title', 'Original Price', 'Discount Price', 'Discount', 'Seller Name', 'Number of Ratings', 'Positive Seller Ratings', 'Ship On Time', 'Chat Response Rate', 'Delivery Type', 'Cash On Delivery', 'Flagship Store']


## **Changing column value from Link to Name**

### **A DataFrame containing the Sub-Category Link and related Name**

In [9]:
sub_categories = pd.read_csv("/kaggle/input/daraz-11-11-top-selling-product-data/subcategories.csv")

### **Viewing Sub-Category**

In [10]:
sub_categories.head()

,Category,SubCategory Name,SubCategory Link
0,Women's & Girls' Fashion,Traditional Clothing,https://www.daraz.com.bd/womens-traditional-cl...
1,Women's & Girls' Fashion,Muslim Wear,https://www.daraz.com.bd/womens-abayas-and-hij...
2,Women's & Girls' Fashion,Clothing,https://www.daraz.com.bd/womens-clothing/?serv...
3,Women's & Girls' Fashion,"Lingerie, Sleep & Lounge",https://www.daraz.com.bd/womens-lingerie-sleep...
4,Women's & Girls' Fashion,Shoes,https://www.daraz.com.bd/womens-shoes/?price=5...


### **Implementing Change**

In [11]:
for i in range(len(data_frame)):
  changed = 0
  j = 0
  while j < len(sub_categories) and changed != 1:
    if data_frame.iloc[i]["SubCategory"] == sub_categories.iloc[j]["SubCategory Link"]:
      data_frame.at[i,'SubCategory'] = sub_categories.iloc[j]["SubCategory Name"]
      changed = 1
    j += 1

### **Viewing Change**

In [12]:
data_frame["SubCategory"]

0                                                  Audio
1                                                  Audio
2                                                  Audio
3                                                  Audio
4                                                  Audio
                              ...                       
13742    Branded Ladies/Girl's Watches Collection Online
13743    Branded Ladies/Girl's Watches Collection Online
13744    Branded Ladies/Girl's Watches Collection Online
13745    Branded Ladies/Girl's Watches Collection Online
13746    Branded Ladies/Girl's Watches Collection Online
Name: SubCategory, Length: 13747, dtype: object

## **Checking the datatype of the columns**

In [13]:
data_frame.dtypes

Category                    object
SubCategory                 object
Title                       object
Original Price               int64
Discount Price               int64
Discount                   float64
Seller Name                 object
Number of Ratings            int64
Positive Seller Ratings      int64
Ship On Time                 int64
Chat Response Rate           int64
Delivery Type               object
Cash On Delivery            object
Flagship Store              object
dtype: object

## **Checking NaN values**

In [14]:
data_frame.isna().sum()

Category                   0
SubCategory                0
Title                      0
Original Price             0
Discount Price             0
Discount                   0
Seller Name                0
Number of Ratings          0
Positive Seller Ratings    0
Ship On Time               0
Chat Response Rate         0
Delivery Type              0
Cash On Delivery           0
Flagship Store             0
dtype: int64

### **We have not any NaN value in any of the column.**

## **Handling redundant symbols from Object type columns**

In [15]:
str_columns = []
for column in columns:
  if str(data_frame[column].dtype) == 'object':
    str_columns.append(column)
    for i in range(len(data_frame)):
      data_frame.at[i, column] = data_frame.iloc[i][column].strip().replace('"',"")

## **Viewing Change**

In [16]:
data_frame[str_columns]

,Category,SubCategory,Title,Seller Name,Delivery Type,Cash On Delivery,Flagship Store
0,Electronic Accessories,Audio,QKZ DM10 Zinc Alloy HiFi Earphone In Ear Earphone,Time Square,Standard Delivery,Cash on Delivery Available,Yes
1,Electronic Accessories,Audio,Uiisii HM 12 Super Bass Stereo In-Ear Earphone...,Subaita Dreamland,Free Delivery,Cash on Delivery Available,No
2,Electronic Accessories,Audio,Uiisii C100 Super Bass Stereo In Ear Headphone...,Time Square,Standard Delivery,Cash on Delivery Available,Yes
3,Electronic Accessories,Audio,Orignal Creative High Bass Audio Earphone with...,Rimjhim Shop,Free Delivery,Cash on Delivery Available,No
4,Electronic Accessories,Audio,Vivo In Year Yearphone is the best sound quali...,Prima Store,Free Delivery,Cash on Delivery Available,No
...,...,...,...,...,...,...,...
13742,Women's & Girls' Fashion,Branded Ladies/Girl's Watches Collection Online,SKMEI Fashion Ladies Casual Elegant Alloy Wate...,SKMEI Flagship Store,Free Delivery,Cash on Delivery Available,No
13743,Women's & Girls' Fashion,Branded Ladies/Girl's Watches Collection Online,SKMEI 9264 Trend Light Luxury Women's Quartz W...,M M Rahman Gallery,Free Delivery,Cash on Delivery Available,Yes
13744,Women's & Girls' Fashion,Branded Ladies/Girl's Watches Collection Online,Carat Round Dial Quartz Watch Exquisite Durabl...,Carat,Free Delivery,Cash on Delivery Available,No
13745,Women's & Girls' Fashion,Branded Ladies/Girl's Watches Collection Online,LIEBIG Women Watches Casual Fashion Date Stain...,LIEBIG,Free Delivery,Cash on Delivery Available,No


In [17]:
data_frame['Category'].nunique()

11

In [18]:
data_frame['Cash On Delivery'].nunique()

7

In [19]:
data_frame['Delivery Type'].nunique()

2

In [20]:
data_frame['Delivery Type'].value_counts()

Delivery Type
Free Delivery        9191
Standard Delivery    4556
Name: count, dtype: int64

In [21]:
data_frame['Cash On Delivery'].value_counts()

Cash On Delivery
Cash on Delivery Available           13486
100% Authentic from Trusted Brand      178
This item is non-returnable             45
14 days free & easy return              25
7 Days Returns                           5
Warranty not available                   5
Daraz Verified                           3
Name: count, dtype: int64

In [22]:
data_frame.describe()

,Original Price,Discount Price,Discount,Number of Ratings,Positive Seller Ratings,Ship On Time,Chat Response Rate
count,13747.000000,13747.000000,13747.000000,13747.000000,13747.000000,13747.000000,13747.000000
mean,5726.778643,4651.739652,35.124689,88.626391,79.782134,32.806285,89.386339
std,20095.719351,17656.684912,21.131157,302.547150,22.431275,46.951572,21.608909
min,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,290.000000,163.500000,18.000000,2.000000,82.000000,0.000000,92.000000
50%,699.000000,389.000000,35.000000,12.000000,86.000000,0.000000,97.000000
75%,1799.500000,1117.000000,51.000000,60.000000,89.000000,100.000000,100.000000
max,479900.000000,479900.000000,98.000000,11914.000000,100.000000,100.000000,100.000000


## **Dropping redundant column**

In [23]:
data_frame.drop(['Cash On Delivery'], inplace=True, axis=1)

In [24]:
data_frame.columns

Index(['Category', 'SubCategory', 'Title', 'Original Price', 'Discount Price',
       'Discount', 'Seller Name', 'Number of Ratings',
       'Positive Seller Ratings', 'Ship On Time', 'Chat Response Rate',
       'Delivery Type', 'Flagship Store'],
      dtype='object')

## **If a seller sells 50 products on original price, then how many products with the discount price have to be sold to reach the amount equal to the break-even point?**
### **Original Price X 50 products = Discount Price X No. of Products?**

In [25]:
data_frame['No. of products to be sold'] = (data_frame['Original Price'] * 50.0) / data_frame['Discount Price']
data_frame['No. of products to be sold'] = data_frame['No. of products to be sold'].apply(lambda x: math.ceil(x))

In [26]:
data_frame['No. of products to be sold']

0        105
1        114
2        107
3        120
4        137
        ... 
13742     71
13743    112
13744     81
13745    107
13746     84
Name: No. of products to be sold, Length: 13747, dtype: int64

## **How much the seller have to increase the sell percentage to reach the break even point?**   

In [27]:
data_frame['Sell percentage to increase'] = ((data_frame['No. of products to be sold'] - 50) / 50)*100
data_frame['Sell percentage to increase'] = data_frame['Sell percentage to increase'].apply(lambda x: round(x,2))

In [28]:
data_frame['Sell percentage to increase']

0        110.0
1        128.0
2        114.0
3        140.0
4        174.0
         ...  
13742     42.0
13743    124.0
13744     62.0
13745    114.0
13746     68.0
Name: Sell percentage to increase, Length: 13747, dtype: float64

## **Importing the prepared dataset**

In [29]:
data_frame.to_csv(f'Top_Selling_Product_Data.csv', index=False)